# Debug Qdrant "cleaned_exams" Collection

Notebook này giúp debug và khắc phục lỗi không thể lấy collection "cleaned_exams" từ Qdrant.


In [1]:
import sys
import os

# Thêm thư mục src vào sys.path
sys.path.append(os.path.abspath('../src'))

# Import các module cần thiết
from core.db.qdrant import QdrantDatabaseConnector
from core.config import settings
from qdrant_client.http.exceptions import UnexpectedResponse
import traceback

print("✅ Import thành công!")


2025-08-01T08:17:58.113739Z [info     ] Connection to database with uri: mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set successful [core.logger_utils] cls=/home/dino/Documents/mathpal/src/core/db/mongo.py
2025-08-01T08:18:01.427354Z [info     ] COMET_DEFAULT_WORKSPACE: mathpal  # such as your Comet username [core.logger_utils] cls=core.opik_utils
HTTP Request: GET https://www.comet.com/api/rest/v2/account-details "HTTP/1.1 200 OK"
HTTP Request: GET https://www.comet.com/api/rest/v2/workspaces "HTTP/1.1 200 OK"
2025-08-01T08:18:03.166794Z [warning  ] Could not configure Opik.      [core.logger_utils] cls=/home/dino/Documents/mathpal/src/core/__init__.py
✅ Import thành công!


In [2]:
print("=== Cấu hình Qdrant ===")
print(f"USE_QDRANT_CLOUD: {settings.USE_QDRANT_CLOUD}")
print(f"QDRANT_DATABASE_HOST: {settings.QDRANT_DATABASE_HOST}")
print(f"QDRANT_DATABASE_PORT: {settings.QDRANT_DATABASE_PORT}")
print(f"QDRANT_CLOUD_URL: {settings.QDRANT_CLOUD_URL}")
print(f"QDRANT_APIKEY: {'***' if settings.QDRANT_APIKEY else 'None'}")


=== Cấu hình Qdrant ===
USE_QDRANT_CLOUD: False
QDRANT_DATABASE_HOST: qdrant
QDRANT_DATABASE_PORT: 6333
QDRANT_CLOUD_URL: 
QDRANT_APIKEY: None


In [3]:
try:
    print("Đang kết nối Qdrant...")
    qdrant_client = QdrantDatabaseConnector()
    print("✅ Kết nối Qdrant thành công!")
    
    # Test kết nối cơ bản
    client_info = qdrant_client._instance.get_collections()
    print(f"✅ Có thể truy cập Qdrant, số collections hiện tại: {len(client_info.collections)}")
    
except Exception as e:
    print(f"❌ Lỗi kết nối Qdrant: {e}")
    print("\nChi tiết lỗi:")
    traceback.print_exc()
    
    print("\n🔧 Các giải pháp:")
    print("1. Kiểm tra Qdrant service có đang chạy không:")
    print("   - Docker: docker-compose up qdrant")
    print("   - Standalone: systemctl status qdrant")
    print("2. Kiểm tra port 6333 có accessible không")
    print("3. Thử kết nối localhost thay vì 'qdrant' host")
    print("4. Chạy: settings.patch_localhost() nếu notebook chạy ngoài container")


Đang kết nối Qdrant...
✅ Kết nối Qdrant thành công!
❌ Lỗi kết nối Qdrant: [Errno -3] Temporary failure in name resolution

Chi tiết lỗi:

🔧 Các giải pháp:
1. Kiểm tra Qdrant service có đang chạy không:
   - Docker: docker-compose up qdrant
   - Standalone: systemctl status qdrant
2. Kiểm tra port 6333 có accessible không
3. Thử kết nối localhost thay vì 'qdrant' host
4. Chạy: settings.patch_localhost() nếu notebook chạy ngoài container


Traceback (most recent call last):
  File "/home/dino/.cache/pypoetry/virtualenvs/mathpal-EU7lYW5B-py3.11/lib/python3.11/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/home/dino/.cache/pypoetry/virtualenvs/mathpal-EU7lYW5B-py3.11/lib/python3.11/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dino/.cache/pypoetry/virtualenvs/mathpal-EU7lYW5B-py3.11/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/home/dino/.cache/pypoetry/virtualenvs/mathpal-EU7lYW5B-py3.11/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dino/.cache/pypoetry/virtualenvs/mathpal-EU7lYW5B-py3.11/lib/python3.11/site-packages/httpcore

In [4]:
# Nếu gặp lỗi kết nối, thử patch localhost
print("=== Thử kết nối với localhost ===")
try:
    settings.patch_localhost()
    print("✅ Đã chuyển sang cấu hình localhost")
    print(f"Host mới: {settings.QDRANT_DATABASE_HOST}")
    
    # Tạo connection mới với localhost
    qdrant_client = QdrantDatabaseConnector()
    client_info = qdrant_client._instance.get_collections()
    print(f"✅ Kết nối localhost thành công! Có {len(client_info.collections)} collections")
    
except Exception as e:
    print(f"❌ Vẫn lỗi với localhost: {e}")
    print("💡 Hãy chạy: docker-compose up -d qdrant")


=== Thử kết nối với localhost ===
✅ Đã chuyển sang cấu hình localhost
Host mới: localhost
HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333/collections "HTTP/1.1 200 OK"
✅ Kết nối localhost thành công! Có 2 collections


In [5]:
try:
    collections = qdrant_client._instance.get_collections()
    print("=== Danh sách Collections trong Qdrant ===")
    
    if not collections.collections:
        print("⚠️ Không có collection nào trong Qdrant")
    else:
        for collection in collections.collections:
            print(f"📁 {collection.name}")
            
    # Kiểm tra xem cleaned_exams có tồn tại không
    collection_names = [c.name for c in collections.collections]
    if "cleaned_exams" in collection_names:
        print("\n✅ Collection 'cleaned_exams' đã tồn tại")
        
        # Lấy thông tin chi tiết
        collection_info = qdrant_client.get_collection("cleaned_exams")
        print(f"📊 Thông tin collection: {collection_info}")
        
    else:
        print("\n❌ Collection 'cleaned_exams' KHÔNG tồn tại")
        print("💡 Sẽ được tạo tự động ở bước tiếp theo")
        
except Exception as e:
    print(f"❌ Lỗi khi kiểm tra collections: {e}")
    traceback.print_exc()


HTTP Request: GET http://localhost:6333/collections "HTTP/1.1 200 OK"
=== Danh sách Collections trong Qdrant ===
📁 vector_exams
📁 cleaned_exams

✅ Collection 'cleaned_exams' đã tồn tại
HTTP Request: GET http://localhost:6333/collections/cleaned_exams "HTTP/1.1 200 OK"
📊 Thông tin collection: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=3 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors={}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_i

In [6]:
try:
    # Thử get collection trước
    try:
        collection_info = qdrant_client.get_collection("cleaned_exams")
        print("✅ Collection 'cleaned_exams' đã tồn tại")
        print(f"📊 Thông tin: {collection_info}")
    except UnexpectedResponse as e:
        if "doesn't exist" in str(e) or "not found" in str(e).lower():
            print("⚠️ Collection 'cleaned_exams' chưa tồn tại, đang tạo...")
            
            # Tạo non-vector collection cho cleaned_exams
            qdrant_client.create_non_vector_collection("cleaned_exams")
            print("✅ Đã tạo collection 'cleaned_exams' thành công!")
            
            # Verify collection đã được tạo
            collection_info = qdrant_client.get_collection("cleaned_exams")
            print(f"✅ Xác nhận collection đã tạo: {collection_info}")
        else:
            raise e
            
except Exception as e:
    print(f"❌ Lỗi khi tạo collection: {e}")
    traceback.print_exc()


HTTP Request: GET http://localhost:6333/collections/cleaned_exams "HTTP/1.1 200 OK"
✅ Collection 'cleaned_exams' đã tồn tại
📊 Thông tin: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=3 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors={}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None, strict_mode_config=

In [7]:
try:
    print("=== Test đọc dữ liệu từ cleaned_exams ===")
    
    # Scroll để lấy dữ liệu
    result = qdrant_client.scroll(collection_name="cleaned_exams", limit=10)
    
    if result[0]:  # Có dữ liệu
        print(f"✅ Tìm thấy {len(result[0])} records trong cleaned_exams")
        
        # Hiển thị mẫu dữ liệu đầu tiên
        if result[0]:
            sample = result[0][0]
            print("\n📋 Mẫu dữ liệu đầu tiên:")
            print(f"ID: {sample.id}")
            print(f"Payload: {sample.payload}")
    else:
        print("⚠️ Collection 'cleaned_exams' trống (chưa có dữ liệu)")
        print("   Điều này bình thường nếu chưa chạy data pipeline")
        print("   Để có dữ liệu: docker-compose up data-crawlers && docker-compose up feature_pipeline")
        
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu từ cleaned_exams: {e}")
    traceback.print_exc()


=== Test đọc dữ liệu từ cleaned_exams ===
HTTP Request: POST http://localhost:6333/collections/cleaned_exams/points/scroll "HTTP/1.1 200 OK"
✅ Tìm thấy 3 records trong cleaned_exams

📋 Mẫu dữ liệu đầu tiên:
ID: 9f43a64b-676c-4268-b8a8-fccd7c2b358b
Payload: {'type': 'exam', 'link': 'https://loigiaihay.com/de-thi-vao-lop-6-mon-toan-truong-nguyen-tat-thanh-nam-2025-co-dap-an-a185630.html', 'cleaned_content': '**Đáp án**\n**HƯỚNG DẪN GIẢI CHI TIẾT**\n**I. TRẮC NGHIỆM**\n**Câu 1****(0,5 điểm)****.** Cho biểu đồ như hình dưới đây. Tìm tỉ số phần trăm số học sinh khối 9 so với số học sinh toàn trường.\n![](https://img.loigiaihay.com/picture/2025/0602/20_1.png)\n**A.** 22,5% **B.** 12,5% **C.** 32% **D.** 25%\n**Cách giải:**\nSố học sinh khối 6 được biểu diễn là 14\nTỉ số phần trăm học sinh khối 9 là: 100% - (12,5% + 40% + 25%) = 22,5%\n**Đáp số: 22,5%**\n**Câu 2****(0,5 điểm)****.** Gieo 1 viên xúc sắc 20 lần liên tiếp kết quả nhận được như bảng sau:\n![](https://img.loigiaihay.com/picture/20

In [8]:
print("=== DEMO: Cách sử dụng cleaned_exams ===")

# Ví dụ code hoàn chỉnh để sử dụng cleaned_exams
def demo_cleaned_exams_usage():
    try:
        # Đảm bảo kết nối localhost (nếu cần)
        settings.patch_localhost()
        
        # Kết nối Qdrant
        qdrant = QdrantDatabaseConnector()
        
        # Kiểm tra collection tồn tại
        try:
            collection_info = qdrant.get_collection("cleaned_exams")
            print("✅ Collection cleaned_exams ready!")
        except:
            # Tạo collection nếu chưa có
            qdrant.create_non_vector_collection("cleaned_exams")
            print("✅ Collection cleaned_exams created!")
        
        # Đọc dữ liệu
        result = qdrant.scroll("cleaned_exams", limit=5)
        
        if result[0]:
            print(f"📊 Có {len(result[0])} records")
            for i, point in enumerate(result[0][:3], 1):
                print(f"Record {i}: ID={point.id}, Type={point.payload.get('type', 'N/A')}")
        else:
            print("⚠️ Collection trống - cần chạy data pipeline")
        
        return True
        
    except Exception as e:
        print(f"❌ Lỗi: {e}")
        return False

# Chạy demo
success = demo_cleaned_exams_usage()
print(f"\n{'🎉 Demo thành công!' if success else '❌ Demo thất bại!'}")


=== DEMO: Cách sử dụng cleaned_exams ===
HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333/collections/cleaned_exams "HTTP/1.1 200 OK"
✅ Collection cleaned_exams ready!
HTTP Request: POST http://localhost:6333/collections/cleaned_exams/points/scroll "HTTP/1.1 200 OK"
📊 Có 3 records
Record 1: ID=9f43a64b-676c-4268-b8a8-fccd7c2b358b, Type=exam
Record 2: ID=c28ee07a-27d7-47f5-bae8-380c5dcaf7a2, Type=exam
Record 3: ID=f4023133-2e19-4284-ae35-1788c53074dc, Type=exam

🎉 Demo thành công!
